In [1]:
import os
import json
import torch
import torchvision.transforms as transforms
from PIL import Image
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
from collections import defaultdict

# Đường dẫn dữ liệu
DATA_PATH = "../data/processed/vqa_data.json"
IMAGE_DIR = "../data/raw/"
PROCESSED_DATA_FILE = "../data/processed/processed_data.pt"

## Tiền xử lý ảnh

In [2]:
print("Bắt đầu tiền xử lý ảnh...")

image_transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        ),  # Chuẩn hóa giống ImageNet
    ]
)

processed_images = {}

with open(DATA_PATH, "r", encoding="utf-8") as f:
    vqa_data = json.load(f)

for item in tqdm(vqa_data, desc="Xử lý ảnh"):
    image_path = os.path.join(IMAGE_DIR, item["image_id"])
    if os.path.exists(image_path):
        try:
            img = Image.open(image_path).convert("RGB")
            img_tensor = image_transform(img)
            processed_images[item["image_id"]] = img_tensor
        except Exception as e:
            print(f"Lỗi xử lý ảnh {image_path}: {e}")
    else:
        print(f"Ảnh không tồn tại: {image_path}")

print(f"Đã xử lý {len(processed_images)} ảnh!\n")

Bắt đầu tiền xử lý ảnh...


Xử lý ảnh: 100%|██████████| 8579/8579 [00:29<00:00, 290.01it/s]

Đã xử lý 8579 ảnh!



## Xây dựng từ điển ký tự (vocab)

In [3]:
print("Xây dựng từ điển ký tự (vocab)...")

vocab = {"<PAD>": 0, "<UNK>": 1}
for item in vqa_data:
    for q in item["questions"]:
        for char in q["question"]:
            if char not in vocab:
                vocab[char] = len(vocab)

vocab_size = len(vocab)
print(f"Kích thước vocab: {vocab_size}\n")

Xây dựng từ điển ký tự (vocab)...
Kích thước vocab: 88



## Xây dựng từ điển câu trả lời (answer_dict)

In [4]:
print("Xây dựng từ điển câu trả lời...")

answer_dict = {"<UNK>": 0}
for item in vqa_data:
    for q in item["questions"]:
        ans = q["correct_answer"]
        if ans not in answer_dict:
            answer_dict[ans] = len(answer_dict)

num_answers = len(answer_dict)
print(f"Số lượng câu trả lời: {num_answers}\n")

Xây dựng từ điển câu trả lời...
Số lượng câu trả lời: 274



## Chia dữ liệu theo folder (đảm bảo cân bằng)

In [7]:
print("Chia dữ liệu đảm bảo cân bằng giữa các tập...")

folder_dict = defaultdict(list)

for item in vqa_data:
    folder_name = item["image_id"].split("/")[0]  # Lấy tên folder (loại quả)
    folder_dict[folder_name].append(item)

train_items, val_items, test_items = [], [], []

for folder, items in folder_dict.items():
    random.shuffle(items)  # Trộn ngẫu nhiên
    train_split = int(0.8 * len(items))
    val_split = int(0.9 * len(items))

    train_items.extend(items[:train_split])
    val_items.extend(items[train_split:val_split])
    test_items.extend(items[val_split:])

print(
    f"Tổng số mẫu sau khi chia: Train: {len(train_items)}, Val: {len(val_items)}, Test: {len(test_items)}\n"
)

Chia dữ liệu đảm bảo cân bằng giữa các tập...
Tổng số mẫu sau khi chia: Train: 6826, Val: 856, Test: 897



##  Mã hóa dữ liệu thành tensor

In [8]:
print("Mã hóa dữ liệu thành tensor...")

def encode_data(items):
    encoded_data = []
    max_len = 30  # Độ dài cố định của câu hỏi

    for item in tqdm(items, desc="Mã hóa dữ liệu"):
        img_tensor = processed_images.get(item["image_id"])
        if img_tensor is None:
            continue  # Bỏ qua ảnh bị thiếu

        for q in item["questions"]:
            # Mã hóa câu hỏi
            question_tensor = torch.tensor(
                [vocab.get(char, vocab["<UNK>"]) for char in q["question"]],
                dtype=torch.long,
            )
            if len(question_tensor) < max_len:
                question_tensor = torch.cat(
                    [
                        question_tensor,
                        torch.tensor(
                            [vocab["<PAD>"]] * (max_len - len(question_tensor))
                        ),
                    ]
                )
            else:
                question_tensor = question_tensor[:max_len]  # Cắt bớt nếu quá dài

            # Mã hóa câu trả lời
            answer = (
                q["correct_answer"] if q["correct_answer"] in answer_dict else "<UNK>"
            )
            answer_id = torch.tensor(answer_dict[answer], dtype=torch.long)

            encoded_data.append(
                {
                    "image": img_tensor,  # Tensor ảnh
                    "question": question_tensor,  # Tensor câu hỏi
                    "answer_id": answer_id,  # ID câu trả lời
                }
            )

    return encoded_data

# Mã hóa train/val/test
train_data = encode_data(train_items)
val_data = encode_data(val_items)
test_data = encode_data(test_items)

print(
    f"Tổng số mẫu dữ liệu sau mã hóa: Train: {len(train_data)}, Val: {len(val_data)}, Test: {len(test_data)}\n"
)

Mã hóa dữ liệu thành tensor...


Mã hóa dữ liệu: 100%|██████████| 897/897 [00:00<00:00, 10398.93it/s]

Tổng số mẫu dữ liệu sau mã hóa: Train: 27304, Val: 3424, Test: 3588



##  Lưu dữ liệu đã xử lý

In [9]:
print("Lưu dữ liệu đã xử lý...")

torch.save(
    {
        "train": train_data,
        "val": val_data,
        "test": test_data,
        "answer_dict": answer_dict,
        "vocab": vocab,
        "vocab_size": vocab_size,
        "metadata": {
            "num_train": len(train_data),
            "num_val": len(val_data),
            "num_test": len(test_data),
            "num_answers": num_answers,
            "num_vocab": vocab_size,
        },
    },
    PROCESSED_DATA_FILE,
)

print(f"Dữ liệu đã lưu thành công tại `{PROCESSED_DATA_FILE}`!\n")

Lưu dữ liệu đã xử lý...
Dữ liệu đã lưu thành công tại `../data/processed/processed_data.pt`!

